# Introduction to Scan (Theano)

To execute a cell: Ctrl-Enter.

The code was executed with the default configuration of Theano: `floatX=float64`, `device=cpu` and the configuration for GPU `floatX=float32,device=cuda`. For additional information about `scan`, please refer to [theano doc](http://deeplearning.net/software/theano/library/scan.html).

Tested with:
- Python 3.6.2, 
- Theano 0.10.0beta1.dev,
- Lasagne 0.2.dev1,
- cuDNN version 6021,
- GeForce GTX TITAN Black

In [6]:
import os
#os.environ['THEANO_FLAGS'] = 'floatX=float64,device=cpu,mode=FAST_RUN'
os.environ['THEANO_FLAGS'] = 'floatX=float32, device=cuda, mode=FAST_RUN'

In [7]:
import numpy as np
import theano
import theano.tensor as T

# Computation graph and loop

Let `A` be a tensor and `k` a positive integer, we are interested by computing `A.^k` (element-wise power). This computation involves a for loop that iterates `k` times. To define the associated computation graph, we use the `scan` function. The arguments of the `scan` function are the:
- fn: a Theano function representing the computation that is done in a single iteration of the loop represented by the scan op. Note that **the order of parameters is fixed by** `scan`: the output of the prior call to `fn` is the first parameter, followed by all non-sequences,
- output_info: initial value of the output variable,
- sequence: A sequence is a Theano variable which Scan will iterate over and give sub-elements to its inner function as input. A sequence has no associated output. For a sequence variable X, at timestep t, the inner function will receive as input the sequence element X[t],
- non_sequences: A non-sequence is a Theano variable which Scan will provide as-is to its inner function. Like a sequence, a non-sequence has no associated output. For a non-sequence variable X, at timestep t, the inner function will receive as input the variable X,
- n_steps: number of iteration

Then, `scan` returns a tuple containing our result and a dictionary of updates (empty in the following case).

In [8]:
k = T.iscalar("k")
A = T.vector("A")

# Symbolic description of the result
result, updates = theano.scan(fn=lambda prior_result, A: prior_result * A,
                              outputs_info=T.ones_like(A),
                              non_sequences=A,
                              n_steps=k)

# We only care about A**k, but scan has provided us with A**1 through A**k.
# Discard the values that we don't care about. Scan is smart enough to
# notice this and not waste memory saving them.
final_result = result[-1]

# compiled function that returns A**k
power = theano.function(inputs=[A,k], outputs=final_result, updates=updates)

print(power(range(10),2))
print(power(range(10),4))

[  0.   1.   4.   9.  16.  25.  36.  49.  64.  81.]
[  0.00000000e+00   1.00000000e+00   1.60000000e+01   8.10000000e+01
   2.56000000e+02   6.25000000e+02   1.29600000e+03   2.40100000e+03
   4.09600000e+03   6.56100000e+03]


With scan, it is important to distinguish the following types of output variables (tap refers to time slices of sequences or outputs):
- Nitsot (no input tap, single output tap) : A nitsot is an output variable of the inner function that is not fed back as an input to the next iteration of the inner function. Nitsots are typically encountered in situations where Scan is used to perform a ‘map’ operation (every element in a tensor is independently altered using a given operation to produce a new tensor) such as squaring every number in a vector,
- Sitsot (single input tap, single output tap) : A sitsot is an output variable of the inner function that is fed back as an input to the next iteration of the inner function. A typical setting where a sitsot might be encountered is the case where Scan is used to compute the cumulative sum over the elements of a vector and a sitsot output is employed to act as an accumulator,
- Mitsot (multiple input taps, single output tap) : A mitsot is an output variable of the inner function that is fed back as an input to future iterations of the inner function (either multiple future iterations or a single one that isn’t the immediate next one). For example, a mitsot might be used in the case where Scan is used to compute the Fibonacci sequence, one term of the sequence at every timestep, since every computed term needs to be reused to compute the two next terms of the sequence,
- Mitmot (multiple input taps, multiple output taps) : These outputs exist but they cannot be directly created by the user. They can appear in a theano graph as a result of taking the gradient of the output of a Scan with respect to its inputs: This will result in the creation of a new scan node used to compute the gradients of the first scan node. If the original Scan had sitsots or mitsots variables, the new Scan will use mitmots to compute the gradients through time for these variables.

The first three types can be used by the user, while the last one is only used internally for computing the gradient through `scan`.
In the next example, we calculate the polynomial by first generating each of the coefficients, and then summing them at the end. There is no accumulation of results so we can set `outputs_info` to `None`. This indicates to scan that it doesn’t need to pass the prior result to `fn`.

The general order of function parameters to fn is:
- sequences (if any),
- prior result(s) (if needed), 
- non-sequences (if any).


In [9]:
coefficients = theano.tensor.vector("coefficients")
x = T.scalar("x")

max_coefficients_supported = 10000

# Generate the components of the polynomial
components, updates = theano.scan(fn=lambda coefficient, power, free_variable: coefficient * (free_variable ** power),
                                  outputs_info=None,
                                  sequences=[coefficients, theano.tensor.arange(max_coefficients_supported)],
                                  non_sequences=x)
# Sum them up
polynomial = components.sum()

# Compile a function
calculate_polynomial = theano.function(inputs=[coefficients, x], outputs=polynomial)

# Test
test_coefficients = np.asarray([1, 0, 2], dtype=np.float32)
test_value = 3
print(calculate_polynomial(test_coefficients, test_value))
print(1.0 * (3 ** 0) + 0.0 * (3 ** 1) + 2.0 * (3 ** 2))

19.0
19.0
